# Should change Tkf below

In [1]:
import importlib
import time
import os
import pickle
import numpy as np
import scipy as sp
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from scipy import ndimage
from scipy import integrate
from scipy import interpolate

# Should change Tkf below

In [2]:
from multi_loc import covariance, assimilate, utilities

In [3]:
%matplotlib inline

In [4]:
def imshow(X, title=None, vmin=None, vmax=None, cmap=None):
    plt.figure()
    im = plt.imshow(X, vmin=vmin, vmax=vmax, cmap=cmap)
    plt.colorbar(im)
    plt.title(title)

In [5]:
tune_name = 'fine_obs'

save_dir = 'RK_tune'


save_dir = os.path.join(
    '/home/travisharty/Documents/python_code/multi_loc/notebooks/',
    save_dir)
    
this_path = os.path.join(save_dir, 'Z_GT.npy')
Zgt = np.load(this_path)

this_path = os.path.join(save_dir, 'Z0_ens.npy')
Z0ens = np.load(this_path)

this_path = os.path.join(save_dir, 't_gt.npy')
t_gt = np.load(this_path)

this_path = os.path.join(save_dir, 't_lr.npy')
t_lr = np.load(this_path)

this_path = os.path.join(save_dir, 'run_vars.pkl')
with open(this_path, mode='rb') as f:
    run_vars = pickle.load(f)

In [6]:
for key, value in run_vars.items():
    print(key + ' = ' + str(value))
    exec(key + '=value')

Nz = 960
K = 32
I = 12
F = 15.0
b = 10.0
c = 2.5
alpha = 0.12414383561643835
beta = 0.013746194824961947
coarse = 8
Nez = 20
Nex = 500
dt_rk = 0.004166666666666667
dt_lr = 0.1
dt_gt = 0.0020833333333333333
Tic = 24.0
Tlr = 4000
Tgt = 250


# Should change Tkf below

In [14]:
tune_dir = os.path.join(save_dir, tune_name)
# if not os.path.exists(tune_dir):
#     os.mkdir(tune_dir)

# dt_trial_rk = 0.05/24
# dt_kf = 0.05
# dt_save = None
# Tkf = Tgt/5     ######## Change this (I forgot) #######


# alpha_range = np.array([0, 0.01, 0.02, 0.04, 0.08], dtype='float')
# loc_range = np.array([5, 10, 15, 20, 30, 40, 50, 60], dtype='float')


# R_sd = 2
# everyH = 24
# Hz = np.eye(Nz)[::everyH]
# Ny = Hz.shape[0]
# Rz = np.eye(Ny) * R_sd**2

# fine_obs_dict = {'dt_trial_rk': dt_trial_rk,
#                  'dt_kf': dt_kf,
#                  'dt_save': dt_save,
#                  'Tkf': Tkf,
#                  'Tspin': Tspin,
#                  'alpha_range': alpha_range,
#                  'loc_range': loc_range,
#                  'R_sd': R_sd,
#                  'everyH': everyH,
#                  'Hz': Hz,
#                  'Rz': Rz}

this_path = os.path.join(tune_dir, 'fine_obs_dict.pkl')
with open(this_path, mode='rb') as f:
    fine_obs_dict = pickle.load(f)

In [16]:
for key, value in fine_obs_dict.items():
    print(key + ' = ' + str(value))
    exec(key + '=value')

dt_trial_rk = 0.0020833333333333333
dt_kf = 0.05
dt_save = None
Tkf = 50.0
Tspin = 25.0
alpha_range = [0.   0.01 0.02 0.04 0.08]
loc_range = [ 5. 10. 15. 20. 30. 40. 50. 60.]
R_sd = 2
everyH = 24
Hz = [[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Rz = [[4. 0. 0. ... 0. 0. 0.]
 [0. 4. 0. ... 0. 0. 0.]
 [0. 0. 4. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 4. 0. 0.]
 [0. 0. 0. ... 0. 4. 0.]
 [0. 0. 0. ... 0. 0. 4.]]


In [18]:
Tspin_index = 500
Tspin = dt_kf * Tspin_index

In [19]:
Nobs = int(Tkf/dt_kf) + 1
t_kf = np.linspace(0, Tkf, Nobs)

Ngt = int(Tgt/dt_gt) + 1
t_gt = np.linspace(0, Tgt, Ngt)
index_Tkf = int(Tkf/dt_gt) + 1

every_t = int(dt_kf/dt_gt)

if not np.allclose(t_kf, t_gt[0:index_Tkf:every_t]):
    print('They do not match')

In [20]:
Zloc = np.arange(Nz)

Zgt_ts = xr.DataArray(Zgt[:, 0:index_Tkf:every_t],
                      dims=('loc', 'time'),
                      coords={'loc': Zloc,
                              'time': t_kf})
Zobs_ts = Zgt[::everyH, 0:index_Tkf:every_t]
Zobs_ts = xr.DataArray(Zobs_ts,
                       dims=('loc', 'time'),
                       coords={'loc': Zloc[::everyH],
                               'time': t_kf})
del Zgt

In [21]:
smooth_range = np.array([4, 8, 16, 32])
coarse_loc_range = np.array([60, 80, 100, 120])
fine_loc_range = np.array([4, 8, 16, 32])
this_alpha = 0.06
alpha_count = 0
N_laml = 20

In [22]:
print(smooth_range.size * coarse_loc_range.size * fine_loc_range.size)

64


In [27]:
importlib.reload(assimilate)
importlib.reload(utilities)


rmse_array_f = np.ones([alpha_range.size,
                        fine_loc_range.size,
                        coarse_loc_range.size]) * np.nan
std_array_f = rmse_array_f.copy()

rmse_array_a = rmse_array_f.copy()
std_array_a = rmse_array_f.copy()

smooth_dir = os.path.join(tune_dir, 'smooth_loc')
if not os.path.exists(smooth_dir):
    os.mkdir(smooth_dir)

for smooth_count, this_smooth in enumerate(smooth_range):
    for fine_loc_count, this_fine_loc in enumerate(fine_loc_range):
        for coarse_loc_count, this_coarse_loc in enumerate(coarse_loc_range):
            this_folder_name = f'inf_{alpha_count:04}_smooth_{smooth_count:04}'
            this_folder_name += f'_floc_{fine_loc_count:04}_cloc_{coarse_loc_count:04}'
            this_dir_name = os.path.join(smooth_dir, this_folder_name)
            if not os.path.exists(this_dir_name):
                os.mkdir(this_dir_name)
            print('alpha: ', this_alpha, '; smooth: ', this_smooth)
            print('fine_loc: ', this_fine_loc, '; coarse_loc: ', this_coarse_loc)
            print(this_dir_name)
            
            rho_Zf = np.arange(Nz)
            rho_Zf = np.minimum(np.abs(Nz - rho_Zf)%Nz, rho_Zf)
            rho_Zf = np.exp(-(rho_Zf**2)/(2*this_fine_loc**2))
            rho_Zf = sp.linalg.circulant(rho_Zf)

            rho_Zc = np.arange(Nz)
            rho_Zc = np.minimum(np.abs(Nz - rho_Zc)%Nz, rho_Zc)
            rho_Zc = np.exp(-(rho_Zc**2)/(2*this_coarse_loc**2))
            rho_Zc = sp.linalg.circulant(rho_Zc)

            this_path = os.path.join(this_dir_name, 'rho0f')
            np.save(this_path, this_fine_loc)
            
            this_path = os.path.join(this_dir_name, 'rho0c')
            np.save(this_path, this_coarse_loc)

            this_path = os.path.join(this_dir_name, 'alpha')
            np.save(this_path, this_alpha)
            
            this_path = os.path.join(this_dir_name, 'smooth')
            np.save(this_path, this_smooth)
            
            this_path = os.path.join(this_dir_name, 'N_laml')
            np.save(this_path, N_laml)

            cycle_dict = {'Z0ens': Z0ens,
                          'Zobs_ts': Zobs_ts,
                          'Rz': Rz, 'Hz': Hz,
                          'Tkf': Tkf,
                          'dt_kf': dt_kf,
                          'dt_rk': dt_trial_rk,
                          'rho_Zc': rho_Zc,
                          'rho_Zf': rho_Zf,
                          'N_laml': N_laml,
                          'smooth_len': this_smooth,
                          'alpha': this_alpha}
            try:
                returned_dict = assimilate.cycle_KF_LM3_smooth(**cycle_dict)
                mu_f = returned_dict['mu_f']
                std_f = returned_dict['std_f']

                mu_a = returned_dict['mu_a']
                std_a = returned_dict['std_a']

                this_path = os.path.join(this_dir_name, 'mu_f.nc')
                mu_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_f.nc')
                std_f.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'mu_a.nc')
                mu_a.to_netcdf(this_path)

                this_path = os.path.join(this_dir_name, 'std_a.nc')
                std_a.to_netcdf(this_path)


                rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_f - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt(
                    ((mu_a - Zgt_ts.sel(time=mu_f.time))**2).mean(axis=0))
                                              .isel(time=slice(Tspin_index, None)).mean())
                std_array_f[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_f**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                std_array_a[alpha_count, fine_loc_count, coarse_loc_count] = (np.sqrt((std_a**2).mean(axis=0))
                                                      .isel(time=slice(Tspin_index, None)).mean())
                print('rmse_f: ', rmse_array_f[alpha_count, fine_loc_count, coarse_loc_count])
                print('rmse_a: ', rmse_array_a[alpha_count, fine_loc_count, coarse_loc_count])
            
            except:
                print(this_dir_name, ' has failed.')

alpha:  0.06 ; smooth:  4
fine_loc:  4 ; coarse_loc:  60
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0000_floc_0000_cloc_0000
rmse_f:  0.5038749781806868
rmse_a:  0.4614715485675911
alpha:  0.06 ; smooth:  4
fine_loc:  4 ; coarse_loc:  80
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0000_floc_0000_cloc_0001
rmse_f:  0.498250190343197
rmse_a:  0.4566088971901612
alpha:  0.06 ; smooth:  4
fine_loc:  4 ; coarse_loc:  100
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0000_floc_0000_cloc_0002
rmse_f:  0.49180692449400215
rmse_a:  0.4513515027723779
alpha:  0.06 ; smooth:  4
fine_loc:  4 ; coarse_loc:  120
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0000_floc_0000_cloc_0003
rmse_f:  0.5045388726792938
rmse_a:  0.4610317337133655
alpha:  0.06 ; smooth:  4
fine

rmse_f:  0.5146092183476014
rmse_a:  0.47890051723016097
alpha:  0.06 ; smooth:  16
fine_loc:  4 ; coarse_loc:  100
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0002_floc_0000_cloc_0002
rmse_f:  0.5133269174647873
rmse_a:  0.47742058537776916
alpha:  0.06 ; smooth:  16
fine_loc:  4 ; coarse_loc:  120
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0002_floc_0000_cloc_0003
rmse_f:  0.5136347954697585
rmse_a:  0.47907989978614274
alpha:  0.06 ; smooth:  16
fine_loc:  8 ; coarse_loc:  60
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0002_floc_0001_cloc_0000
rmse_f:  0.49321838524588124
rmse_a:  0.4580330457597313
alpha:  0.06 ; smooth:  16
fine_loc:  8 ; coarse_loc:  80
/home/travisharty/Documents/python_code/multi_loc/notebooks/RK_tune/fine_obs/smooth_loc/inf_0000_smooth_0002_floc_0001_cloc_0001
rmse_f:  0.49451030746

In [29]:
this_path = os.path.join(smooth_dir, 'rmse_array_f.npy')
np.save(this_path, rmse_array_f)

this_path = os.path.join(smooth_dir, 'std_array_f.npy')
np.save(this_path, std_array_f)

this_path = os.path.join(smooth_dir, 'rmse_array_a.npy')
np.save(this_path, rmse_array_a)

this_path = os.path.join(smooth_dir, 'std_array_a.npy')
np.save(this_path, std_array_a)